#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 6, 20)

### Begin of Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name qty target active spd current change percent reason market xdate'.split()

format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,TTB,"100,000",1.27,2,CP1S,SET50,2022-04-21


### Print to verify before upload file

In [3]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active, xdate
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

#df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab = pn.widgets.Tabulator(orders)
df_tab

Tabulator(value=   trade    name  ...)

In [4]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

### Process to set target price

#### 1) Set50 records

In [5]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,5,B,GLOBAL,9000,18.40,1,3L,SET50,2022-02-02
1,6,B,IVL,3000,46.25,1,P50%3M,SET50,2022-05-31
2,8,B,KTC,2100,55.00,0,P25%,SET50,2022-04-18
3,12,S,BDMS,1500,26.25,0,CP1S,SET50,2022-03-09
4,17,S,SCC,300,400.00,0,C398,SET50,2022-04-07
5,19,S,TISCO,1000,97.75,0,CP1S,SET50,2022-04-29
6,20,S,TTB,100000,1.27,2,CP1S,SET50,2022-04-21


In [6]:
name = 'KTC'
limit = 60 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,59.78,60.43,59.15,6279878.22,59.78
std,1.76,1.72,1.89,4059438.82,1.91
min,57.00,57.50,55.00,1229623.00,56.25
25%,58.25,59.12,57.50,3729561.75,58.25
50%,59.75,60.38,59.12,5006019.00,59.75
75%,61.25,61.75,60.75,7164547.50,61.25
max,63.25,64.00,63.25,19099266.00,63.75


#### 2) Set100 records

In [7]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,AP,10000,9.5,1,MinP,SET100,2022-05-10
1,3,B,BCPG,12000,11.0,1,MinP,SET100,2022-02-02
2,4,B,BPP,9000,14.0,1,MinP,SET100,2022-02-02
3,7,B,KKP,3000,62.0,1,MinP,SET100,2022-04-28
4,11,S,BCH,3000,21.0,0,5pct,SET100,2022-05-10
5,13,S,DOHOME,2400,20.3,0,C20.25,SET100,2022-02-02


In [8]:
name = 'SINGER'
limit = 60 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,60.00,60.00,60.00,60.00,60.00
mean,52.40,53.30,51.50,4814272.83,52.35
std,2.97,2.84,2.89,3673494.73,2.97
min,47.00,48.75,46.25,1337690.00,47.25
25%,50.00,50.75,49.44,2439517.75,50.00
50%,51.88,53.00,51.12,3978535.00,51.88
75%,54.75,55.75,53.81,5677816.50,54.75
max,58.75,59.25,57.50,23670812.00,59.00


#### 3) Set records

In [9]:
# Why SIS does not print out from this SQL?
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,ASK,3000,32.00,0,MinP,SET,2022-03-03
1,9,B,SSP,10000,9.00,0,6L,SET,2022-02-02
2,10,B,TMT,12000,8.40,1,RD15pct,SET,2022-04-18
3,14,S,JASIF,10000,10.60,2,C10.10,SET,2022-05-20
4,15,S,MAKRO,1500,39.00,0,6M,SET,2022-03-03
5,16,S,NER,9000,7.10,0,P50%,SET,2022-04-19
6,18,S,SIS,3600,28.75,1,C28.50,SET,2022-03-10


In [10]:
name = 'ASK'
limit = 120 # 1 month of data
sql = """
SELECT P.*
FROM price P
WHERE P.name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe().round(2)

,price,maxp,minp,qty,opnp
count,120.00,120.00,120.00,120.00,120.00
mean,43.93,44.74,43.17,1471663.27,44.04
std,3.03,2.92,3.03,1121675.82,2.98
min,33.25,34.00,32.75,187563.00,33.00
25%,42.44,43.19,41.69,697118.00,42.50
50%,44.50,45.00,44.00,1218279.00,44.50
75%,46.00,46.75,45.00,1833162.00,46.06
max,49.00,49.50,48.25,6882222.00,49.25


In [11]:
pd.read_sql_query('SELECT * FROM orders ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,AP,10000,9.50,1,MinP,SET100,2022-05-10
1,2,B,ASK,3000,32.00,0,MinP,SET,2022-03-03
2,3,B,BCPG,12000,11.00,1,MinP,SET100,2022-02-02
3,4,B,BPP,9000,14.00,1,MinP,SET100,2022-02-02
4,5,B,GLOBAL,9000,18.40,1,3L,SET50,2022-02-02
5,6,B,IVL,3000,46.25,1,P50%3M,SET50,2022-05-31
6,7,B,KKP,3000,62.00,1,MinP,SET100,2022-04-28
7,8,B,KTC,2100,55.00,0,P25%,SET50,2022-04-18
8,9,B,SSP,10000,9.00,0,6L,SET,2022-02-02
9,10,B,TMT,12000,8.40,1,RD15pct,SET,2022-04-18
